# 1 流程分析

之前无意中在这个网站发现有各个高校的历年录取分数线：[](https://gkcx.eol.cn/)[https://gkcx.eol.cn](https://gkcx.eol.cn/)。

我们的目标是用 Python 将下面页面的数据导出到 Excel：

![](http://ww3.sinaimg.cn/large/006tNc79ly1g53u4ssdqjj30m109nq44.jpg)

这个页面的 URL 是：[](https://gkcx.eol.cn/schoolhtm/schoolTemple/school160.htm)[https://gkcx.eol.cn/schoolhtm...](https://gkcx.eol.cn/schoolhtm/schoolTemple/school160.htm)，显然是需要一个 `school_id` 拼接而成的，那么如何获取这个 school_id 呢？

除非想办法爬取到所有院校的 school_id，这里我想着是从上面图中的搜索框进入：

![](http://ww4.sinaimg.cn/large/006tNc79ly1g53u57igvlj30m80g876x.jpg)

这样，整体的业务流程我们就理清楚了：

* 先调用搜索的 URL 获取到高校的 school_id，拼接到高校的详情访问地址
* 访问详情地址，抓取目标数据
* 处理目标数据，存储到 Excel 中

# 2 获取 school_id

按下 `F12`，可以看出搜索调用的 URL 是：`https://gkcx.eol.cn/soudaxue/queryschool.html?&keyWord1=南京邮电大学`，但是我们发现该请求的 response 里并没有高校列表，所以猜测这里是有二次数据请求获取到高校的列表，然后解析显示到页面的。

顺着请求流，我们看到了这么一个请求：

![](http://ww4.sinaimg.cn/large/006tNc79ly1g53u5l4l1rj30m808x75h.jpg)

并且它的 response 刚好是一个包含高校信息的 json，到这里应该还是顺利的，我们只要从这个 json 里解析出我们想要的东西，然后继续后面的步骤就可以了。要注意该请求的 `Referer`。

但是在解析这个 json 时会遇到一个小问题，返回的数据格式是这样的：


```json
({
 "totalRecord": {"num": "2"},
 "school":  [
    {
   "schoolid": "160",
   "schoolname": "南京邮电大学",
...
});
```

它是被 `();` 包围着的，不是一个合法的 json 数据，这里需要对其进行处理后才能解析 json：


```python
# 返回数据包含 ();，需要特殊处理
    text = ((response.text).split(');',1)[0]).split('(',1)[1]
    j = json.loads(text)
```

# 3 分数线获取

学校的详情页面是：[](https://gkcx.eol.cn/schoolhtm/schoolTemple/school160.htm)[https://gkcx.eol.cn/schoolhtm...](https://gkcx.eol.cn/schoolhtm/schoolTemple/school160.htm)，同样的套路，在点击后 response 里并没有分数线数据，我想也是二次请求吧，果然在请求流里找到了这个：

![](http://ww4.sinaimg.cn/large/006tNc79ly1g53u5vv4avj30m80ivgpf.jpg)

这里的两个请求刚好将高校的每年分数线和各专业的分数线以 XML 的格式返回，Very Good！

下面要做的就是 XML 解析啦。

# 4 XML 解析

这里我们使用 `xml.etree.ElementTree` 来解析 XML：


```xml
<areapionts>
    <areapiont>
        <year>2017</year>
        <specialname>软件工程（嵌入式培养）</specialname>
        <maxfs>369</maxfs>
        <varfs>366</varfs>
        <minfs>364</minfs>
        <pc>一批</pc>
        <stype>理科</stype>
    </areapiont>
```

由于数据比较规整，解析也很简单：


```python
areapionts = ET.fromstring(response.text)
for areapiont in areapionts:
    print(areapiont.find('year').text)
    print(areapiont.find('specialname').text)
```

# 5 Excel 写入

Excel 的写入需要借助于 `openpyxl` 模块。

* openpyxl 简单使用示例


```python
>>> import openpyxl
>>> wb = openpyxl.Workbook()
# 初始时会生成一个 sheet 页
>>> wb.sheetnames
['Sheet']
# 创建 sheet 页
>>> wb.create_sheet(index=0,title='First')
<Worksheet "First">
# 获取所有 sheet 页
>>> wb.sheetnames
['First', 'Sheet']
# 删除 sheet 页
>>> wb.remove(wb['Sheet'])
>>> wb.sheetnames
['First']
>>> sheet = wb['First']
# 设置单元格
>>> sheet['A1'] = '省份'
>>> sheet['B1'] = '学校'
# 设置指定的单元格
>>> sheet.cell(1,3).value='test'
>>> wb.save('test.xlsx')
```

* XML 解析写入 Excel


```python
def gen_excel(school,xml,wb):
    sheet = wb.create_sheet(title='各专业历年录取分数线')
    sheet.column_dimensions['B'].width = 40
    sheet['A1'] = '年份'
    sheet['B1'] = '专业'
    sheet['C1'] = '最高分'
    sheet['D1'] = '平均分'
    sheet['E1'] = '最低分'
    sheet['F1'] = '批次'
    sheet['G1'] = '录取批次'

    areapionts = ET.fromstring(xml)
    column = 1 
    for areapiont in areapionts:
        column += 1
        sheet.cell(column,1).value = areapiont.find('year').text
        sheet.cell(column,2).value = areapiont.find('specialname').text
        sheet.cell(column,3).value = areapiont.find('maxfs').text
        sheet.cell(column,4).value = areapiont.find('varfs').text
        sheet.cell(column,5).value = areapiont.find('minfs').text
        sheet.cell(column,6).value = areapiont.find('pc').text
        sheet.cell(column,7).value = areapiont.find('stype').text
    wb.save('{}.xlsx'.format(school['schoolname']))
```

# 执行效果


```sh
$ python gkcx.py
Please the school name：南京邮电大学
共检索到 2 个高校：['南京邮电大学', '南京邮电大学通达学院']
数据获取完成，已下载到脚本目录
```

![](http://ww4.sinaimg.cn/large/006tNc79ly1g53u66ufn5j30m80cu0vl.jpg)

In [3]:

# encoding:utf-8
 
from bs4 import BeautifulSoup
import requests
import json
import re
import openpyxl
import xml.etree.ElementTree as ET
 
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"
}
 
def get_school_id(school_name):
    Referer = "https://gkcx.eol.cn/soudaxue/queryschool.html?&keyWord1={}".format(school_name)
    data_url = "https://data-gkcx.eol.cn/soudaxue/queryschool.html"
    params = {
        "messtype" : "jsonp",
        "_":"1530074932531",
        "callback" : "",
        "keyWord1" : school_name
    }
    headers["Referer"] = Referer.encode('utf-8')
 
    response = requests.request("GET", data_url,headers=headers,params=params)
    # 返回数据包含 ();，需要特殊处理
    text = ((response.text).split(');',1)[0]).split('(',1)[1]
    j = json.loads(text)
    school_list = j['school']
    return [{'schoolid': i['schoolid'], 'schoolname':i['schoolname']} for i in school_list]
 
def get_xml(school):
    id = school['schoolid']
    Referer = "https://gkcx.eol.cn/schoolhtm/schoolTemple/school{}.htm".format(id)
    headers["Referer"] = Referer.encode('utf-8')
    data_url = "https://gkcx.eol.cn/commonXML/schoolSpecialPoint/schoolSpecialPoint{}_10014_10035.xml".format(id)
    response = requests.request("GET", data_url, headers=headers)
    response.encoding='utf-8'
    return response.text
 
def gen_excel(school,xml,wb):
    sheet = wb.create_sheet(title='各专业历年录取分数线')
    # 设置专业列的列宽
    sheet.column_dimensions['B'].width = 40
    sheet.freeze_panes = 'A1'
    sheet['A1'] = '年份'
    sheet['B1'] = '专业'
    sheet['C1'] = '最高分'
    sheet['D1'] = '平均分'
    sheet['E1'] = '最低分'
    sheet['F1'] = '批次'
    sheet['G1'] = '录取批次'
 
    areapionts = ET.fromstring(xml)
    column = 1
    for areapiont in areapionts:
        column += 1
        sheet.cell(column,1).value = areapiont.find('year').text
        sheet.cell(column,2).value = areapiont.find('specialname').text
        sheet.cell(column,3).value = areapiont.find('maxfs').text
        sheet.cell(column,4).value = areapiont.find('varfs').text
        sheet.cell(column,5).value = areapiont.find('minfs').text
        sheet.cell(column,6).value = areapiont.find('pc').text
        sheet.cell(column,7).value = areapiont.find('stype').text
    wb.save('{}.xlsx'.format(school['schoolname']))
 
if __name__ == "__main__":
    school_name = input("Please the school name：")
    # school_name = '南京邮电大学'
    school_infos = get_school_id(school_name)
    print("共检索到 {} 个高校：{}".format(len(school_infos), [info['schoolname'] for info in school_infos]))
    for school in school_infos:
        wb = None
        try:
            wb = openpyxl.load_workbook('{}.xlsx'.format(school['schoolname']))
        except FileNotFoundError:
            wb = openpyxl.Workbook()
        # 删除之前的 sheet 页
        for sheet in wb.sheetnames:
            wb.remove(wb[sheet])
        xml = get_xml(school)
        gen_excel(school,xml,wb)
    print("数据获取完成，已下载到脚本目录")

Please the school name：南京邮电大学


ConnectionError: HTTPSConnectionPool(host='data-gkcx.eol.cn', port=443): Max retries exceeded with url: /soudaxue/queryschool.html?messtype=jsonp&_=1530074932531&callback=&keyWord1=%E5%8D%97%E4%BA%AC%E9%82%AE%E7%94%B5%E5%A4%A7%E5%AD%A6 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1162ee0f0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))